In [73]:
# installing the Kaggle library
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [74]:
# configuring the path of Kaggle.json file
!mkdir -p ~/.kaggle

In [75]:
!cp kaggle.json ~/.kaggle/

In [76]:
!chmod 600 ~/.kaggle/kaggle.json

In [77]:
# Kaggle api

!kaggle competitions download -c fake-news

fake-news.zip: Skipping, found more recently modified local copy (use --force to force download)


In [78]:
import pandas as pd

In [79]:
!ls

fake-news.zip  kaggle.json  sample_data  submit.csv  test.csv  train.csv


In [80]:
# extracting the compressed dataset
from zipfile import ZipFile

dataset = '/content/fake-news.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The datadet is extracted')

The datadet is extracted


In [81]:
df= pd.read_csv('/content/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [82]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [83]:
len(df)

20800

In [84]:
df = df.dropna()

In [85]:
len(df)

18285

In [86]:
# getting independent features
x= df.drop('label',axis=1)

In [87]:
# dependent feature
y=df['label']

## **Importing libies**

In [88]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [89]:
# vocabularry size
voc_size = 5000

# **One Hot Representtion**

In [90]:
messages = x.copy()

In [91]:
messages.reset_index(inplace = True)

In [92]:
import  nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [93]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [94]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [95]:
from tensorflow.keras.preprocessing.text import one_hot

In [96]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
print(onehot_repr)

[[1503, 4588, 905, 481, 2553, 3506, 1501, 1354, 203, 2345], [748, 2225, 674, 1242, 1097, 323, 2470], [3061, 1059, 823, 1044], [3297, 2986, 4136, 3514, 4612, 958], [4479, 1097, 2916, 4242, 2947, 3152, 1097, 408, 3110, 2374], [2902, 1082, 1818, 1682, 662, 4490, 4698, 3677, 4640, 3930, 2085, 18, 529, 2306, 2470], [3276, 2776, 503, 1701, 1952, 2285, 3829, 319, 2772, 4914, 1759], [4895, 4277, 647, 1618, 1343, 3509, 4490, 4855, 2772, 4914, 1759], [382, 2925, 2573, 4176, 1621, 3313, 962, 2929, 4490, 2892], [3108, 2979, 2058, 2100, 534, 472, 1602, 1546], [2343, 2525, 2686, 4839, 2422, 3019, 3933, 1684, 4323, 4757, 4034], [3514, 1119, 2553, 3313, 4490, 1343], [2004, 4897, 1959, 1390, 4988, 1931, 1081, 2759, 740], [2026, 1048, 3370, 3571, 1439, 1304, 1156, 2772, 4914, 1759], [4938, 1237, 1523, 2070, 795, 2772, 4914, 1759], [4460, 1872, 1384, 2135, 1499, 972, 4104, 371, 201, 238], [2926, 1813, 2225], [329, 4327, 2155, 1612, 4490, 3789, 1181, 2470], [493, 423, 674, 964, 4628, 2563, 2779, 37, 3152]

In [97]:
# choose max sentence length = 20
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding = 'pre',maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ... 1354  203 2345]
 [   0    0    0 ... 1097  323 2470]
 [   0    0    0 ... 1059  823 1044]
 ...
 [   0    0    0 ... 2772 4914 1759]
 [   0    0    0 ... 1152 1729 3590]
 [   0    0    0 ... 4019 3795 3488]]


In [98]:
## creating model
embedding_vector_features= 40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) # making embedding layer
model.add(LSTM(100))  # one LSTM Layer with 100 neurons
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_5 (LSTM)               (None, 100)               56400     
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [99]:
## creating model ( bi directional lstm model)
from tensorflow.keras.layers import Bidirectional

In [100]:
## creating BiLSTM model 1
embedding_vector_features= 40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) # making embedding layer
model1.add(Bidirectional(LSTM(100)))  # one LSTM Layer with 100 neurons
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_2 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [101]:
import numpy as np 
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [102]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33,random_state=0)

In [103]:
model1.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 16ms/step - loss: 0.2970 - accuracy: 0.8609 - val_loss: 0.2014 - val_accuracy: 0.9188
Epoch 2/10
192/192 [==============================] - 2s 11ms/step - loss: 0.1447 - accuracy: 0.9472 - val_loss: 0.2005 - val_accuracy: 0.9206
Epoch 3/10
192/192 [==============================] - 2s 11ms/step - loss: 0.1012 - accuracy: 0.9647 - val_loss: 0.2340 - val_accuracy: 0.9104
Epoch 4/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0756 - accuracy: 0.9727 - val_loss: 0.2885 - val_accuracy: 0.9153
Epoch 5/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0558 - accuracy: 0.9794 - val_loss: 0.2778 - val_accuracy: 0.9107
Epoch 6/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0410 - accuracy: 0.9864 - val_loss: 0.3687 - val_accuracy: 0.9099
Epoch 7/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0265 - accuracy: 0.9916 - val_loss: 0.4078 - val_accuracy: 0.9075

In [104]:
y_pred=model1.predict(x_test)

In [105]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [106]:
from sklearn.metrics import confusion_matrix

In [107]:
confusion_matrix(y_test,y_pred)

array([[3137,  252],
       [ 329, 2317]])

In [108]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.903728251864126

In [109]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.93      0.92      3389
           1       0.90      0.88      0.89      2646

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035



## creating model by adding dropout layer

In [110]:
from tensorflow.keras.layers import Dropout

In [111]:
## Creating BiLSTM model 2 with Dropouts
embedding_vector_features= 40
model2=Sequential()
model2.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) # making embedding layer
model.add(Dropout(0.3))
model2.add(Bidirectional(LSTM(100)))  # one LSTM Layer with 100 neurons
model.add(Dropout(0.3))
model2.add(Dense(1,activation='sigmoid'))
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model2.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_3 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dense_7 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [112]:
import numpy as np
x_finall= np.array(embedded_docs)
y_finall= np.array(y)

In [113]:
x_train,x_test,y_train,y_test=train_test_split(x_finall,y_finall,test_size=0.33,random_state=0)

In [114]:
model2.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 16ms/step - loss: 0.2898 - accuracy: 0.8617 - val_loss: 0.1987 - val_accuracy: 0.9178
Epoch 2/10
192/192 [==============================] - 2s 11ms/step - loss: 0.1396 - accuracy: 0.9476 - val_loss: 0.2183 - val_accuracy: 0.9205
Epoch 3/10
192/192 [==============================] - 3s 15ms/step - loss: 0.1021 - accuracy: 0.9652 - val_loss: 0.2278 - val_accuracy: 0.9193
Epoch 4/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0765 - accuracy: 0.9743 - val_loss: 0.2839 - val_accuracy: 0.9075
Epoch 5/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0602 - accuracy: 0.9810 - val_loss: 0.3203 - val_accuracy: 0.9115
Epoch 6/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0418 - accuracy: 0.9861 - val_loss: 0.3495 - val_accuracy: 0.9099
Epoch 7/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0297 - accuracy: 0.9909 - val_loss: 0.4166 - val_accuracy: 0.9094

In [115]:
y_pred2=model2.predict(x_test)

In [116]:
y_pred2=np.where(y_pred2 > 0.6, 1,0) ##AUC ROC Curve

In [117]:
ac= accuracy_score(y_test,y_pred2)
print(ac)

0.9073736536868269


In [118]:
cm= confusion_matrix(y_test,y_pred2)
print(cm)

[[3128  261]
 [ 298 2348]]


In [119]:
## Creating LSTM model 3
embedding_vector_features=40 ##features representation
model3=Sequential()
model3.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model3.add(LSTM(100))
model3.add(Dense(1,activation='sigmoid'))
model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model3.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_8 (LSTM)               (None, 100)               56400     
                                                                 
 dense_8 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [120]:
len(embedded_docs),y.shape

(18285, (18285,))

In [121]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [122]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [123]:
### Finally Training Lstm model
model3.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 4s 10ms/step - loss: 0.3334 - accuracy: 0.8415 - val_loss: 0.1915 - val_accuracy: 0.9163
Epoch 2/10
192/192 [==============================] - 2s 8ms/step - loss: 0.1407 - accuracy: 0.9444 - val_loss: 0.1940 - val_accuracy: 0.9163
Epoch 3/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0993 - accuracy: 0.9661 - val_loss: 0.1979 - val_accuracy: 0.9211
Epoch 4/10
192/192 [==============================] - 1s 8ms/step - loss: 0.0680 - accuracy: 0.9775 - val_loss: 0.2469 - val_accuracy: 0.9193
Epoch 5/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0455 - accuracy: 0.9847 - val_loss: 0.3295 - val_accuracy: 0.9092
Epoch 6/10
192/192 [==============================] - 1s 8ms/step - loss: 0.0280 - accuracy: 0.9921 - val_loss: 0.3488 - val_accuracy: 0.9153
Epoch 7/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0155 - accuracy: 0.9960 - val_loss: 0.4298 - val_accuracy: 0.9148
Epoch

In [124]:
y_pred=model3.predict(x_test)

In [125]:
y_pred=np.where(y_pred > 0.6, 1,0) 

In [126]:
from sklearn.metrics import confusion_matrix

In [127]:
confusion_matrix(y_test,y_pred)

array([[3096,  293],
       [ 239, 2407]])

In [128]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.911847555923778